In [60]:
from enum import Enum
from dataclasses import dataclass, field

class Button(Enum):
    A = 1
    B = 2

@dataclass()
class Machine:
    button_a: tuple[int, int] = ()
    ratio_a: tuple[int, int] = ()
    button_b: tuple[int, int] = ()
    ratio_b: tuple[int, int] = ()

    prize: tuple[int, int] = ()
    max_a_presses: int = 0
    max_b_presses: int = 0

    def init_maxes(self):
        ax = self.button_a[0]
        ay = self.button_a[1]
        bx = self.button_b[0]
        by = self.button_b[1]

        px = self.prize[0]
        py = self.prize[1]

        max_ax = px // ax
        max_ay = py // ay
        max_bx = px // bx
        max_by = py // by
        self.max_a_presses = min(max_ax, max_ay)
        self.max_b_presses = min(max_bx, max_by)
        self.ratio_a = (self.button_a[0] / self.button_a[1]).as_integer_ratio()
        self.ratio_b = (self.button_b[0] / self.button_b[1]).as_integer_ratio()



def load_input() -> list[Machine]:
    with open("../../data/day13-input.txt") as f:
        machines: list[Machine] = []
        for group in f.read().split('\n\n'):
            machine = Machine()
            rules = group.split('\n')
            parts = rules[0].split(':')[1].replace('X+', '').replace('Y+', '').replace(' ', '').split(',')
            machine.button_a = (int(parts[0]), int(parts[1]))
            parts = rules[1].split(':')[1].replace('X+', '').replace('Y+', '').replace(' ', '').split(',')
            machine.button_b = (int(parts[0]), int(parts[1]))
            parts = rules[2].split(':')[1].replace('X=', '').replace('Y=', '').replace(' ', '').split(',')
            machine.prize = (int(parts[0]), int(parts[1]))
            machine.init_maxes()
            machines.append(machine)
    return machines


In [61]:
class Decision:
    def __init__(self, button: Button, position: tuple[int, int], parent: 'Decision' = None):
        self.button = button
        self.position = position
        self.parent = parent


def is_out_of_bounds(position: tuple[int, int], machine: Machine) -> bool:
    if position[0] > machine.prize[0] or position[1] > machine.prize[1]:
        return True
    return False


def press_button(machine: Machine, position: tuple[int,int], button: Button):
    if button == Button.A:
        next_position = machine.button_a[0] + position[0], machine.button_a[1] + position[1]
    else:
        next_position = machine.button_b[0] + position[0], machine.button_b[1] + position[1]
    return next_position

def proceed(machine: Machine, current_decision: Decision, winners: list[Decision]):
    # Winner
    if current_decision.position == machine.prize:
        return current_decision

    # Push A
    next_for_press_a = press_button(machine, current_decision.position, Button.A)
    if not is_out_of_bounds(next_for_press_a, machine):
        decision = Decision(button=Button.A, position=next_for_press_a, parent=current_decision)
        winner = proceed(machine, decision, winners)
        if winner:
            winners.append(winner)

    # Push B
    next_for_press_b = press_button(machine, current_decision.position, Button.B)
    if not is_out_of_bounds(next_for_press_b, machine):
        decision = Decision(button=Button.B, position=next_for_press_b, parent=current_decision)
        winner = proceed(machine, decision, winners)
        if winner:
            winners.append(winner)

def solve_machine(machine: Machine):
    winners = []
    proceed(machine, Decision(button=Button.A, position=press_button(machine, (0,0), Button.A), parent=None), winners)
    proceed(machine, Decision(button=Button.B, position=press_button(machine, (0,0), Button.B), parent=None), winners)
    return winners

def solve_problem():
    machines = load_input()
    winners = solve_machine(machines[0])
    print(len(winners))

# Don't run this.
# solve_problem()
# After thinking about this problem, I realize that I have to determine if the prize
# is in the span of the two vectors A, B. If so, then determine the cheapest
# linear combination path to it.

In [72]:
def vector_can_hit_target(vector: tuple[int, int], from_vector: tuple[int, int], target_vector: tuple[int, int]) -> bool:
    """If so, then return how many steps it takes"""
    reference_vector = (target_vector[0] - from_vector[0], target_vector[1] - from_vector[1])
    ratio = (reference_vector[0] / reference_vector[1]).as_integer_ratio()
    if ratio == (vector[0] / vector[1]).as_integer_ratio():
        return reference_vector[0] // vector[1]
    return False

def find_steps_to_target(vector: tuple[int, int], target_vector: tuple[int,int], step: int, min_step: int, max_step: int) -> int:
    """ Binary search, but it's not needed"""
    a, b = step * vector[0], step * vector[1]
    if a == target_vector[0] and b == target_vector[1]:
        #print("found:", step)
        return step
    elif a < target_vector[0]:
        # step forward by 1/2
        next_step = step + max(1, (max_step - step) // 2)
        #print("too little:", step, "next = ", next_step)
        return find_steps_to_target(vector, target_vector, next_step, step, max_step)
    else:
        # Step back by 1/2
        next_step = min_step + max(1, (step - min_step) // 2)
        #print("too big:", step, "next = ", next_step)
        return find_steps_to_target(vector, target_vector, next_step, min_step, step)

def solve_machine(machine: Machine):
    vec_a = machine.button_a
    vec_b = machine.button_b
    solutions = []

    # Check if you don't need to press B
    if vector_can_hit_target(vec_a, (0, 0), machine.prize):
        only_a_presses =  vector_can_hit_target(vec_a,(0,0), machine.prize)
        if only_a_presses is not False:
            solutions.append((only_a_presses, 0))

    for a_press in range(0, min(10000, machine.max_a_presses)):
        a_position = a_press * vec_a[0], a_press * vec_a[1]
        steps_to_target = vector_can_hit_target(vec_b, a_position, machine.prize)
        if steps_to_target is not False:
            solutions.append((a_press, steps_to_target))
    return solutions


def solve_problem():
    cost_a = 3
    cost_b = 1

    machines = load_input()
    #machine = machines[2]
    #print("Trying machine:", machine)
    #solution = solve_machine(machine)
    #if solution:
    #    print(solution)
    total_cost = 0
    for machine in machines:
        solution = solve_machine(machine)
        if solution:
            print(solution)
            if len(solution) > 1:
                raise Exception("Not handled")
            a, b = solution[0]
            if 100 < a  or 100 < b:
                print('skip:', solution[0])
                continue
            total_cost += a * cost_a + b * cost_b

    return total_cost
solve_problem()



[(13, 46)]
[(67, 258)]
skip: (67, 258)
[(99, 132)]
skip: (99, 132)
[(63, 18)]
[(37, 4)]
[(47, 144)]
skip: (47, 144)
[(56, 245)]
skip: (56, 245)
[(22, 13)]
[(72, 28)]
[(81, 33)]
[(77, 18)]
[(18, 100)]
[(49, 120)]
skip: (49, 120)
[(9, 42)]
[(19, 12)]
[(66, 86)]
[(71, 72)]
[(21, 8)]
[(10, 4)]
[(79, 159)]
skip: (79, 159)
[(42, 245)]
skip: (42, 245)
[(33, 148)]
skip: (33, 148)
[(8, 14)]
[(53, 241)]
skip: (53, 241)
[(92, 32)]
[(41, 13)]
[(54, 193)]
skip: (54, 193)
[(95, 427)]
skip: (95, 427)
[(90, 20)]
[(57, 102)]
skip: (57, 102)
[(73, 220)]
skip: (73, 220)
[(86, 36)]
[(51, 42)]
[(37, 91)]
[(12, 28)]
[(50, 27)]
[(26, 28)]
[(94, 13)]
[(46, 81)]
[(33, 13)]
[(53, 19)]
[(86, 123)]
skip: (86, 123)
[(67, 27)]
[(66, 109)]
skip: (66, 109)
[(10, 184)]
skip: (10, 184)
[(54, 1)]
[(40, 12)]
[(96, 3)]
[(83, 37)]
[(94, 97)]
[(11, 5)]
[(78, 61)]
[(16, 202)]
skip: (16, 202)
[(92, 76)]
[(45, 105)]
skip: (45, 105)
[(88, 48)]
[(37, 81)]
[(69, 107)]
skip: (69, 107)
[(33, 306)]
skip: (33, 306)
[(33, 15)]
[(30, 5

19384

In [ ]:
#19384 < x < 38195